# Confidential containers on ACI Demo

### Step 1 : Setup 

In this step we will 
- Setup the necessary libraries and customizable variables 
- Request the public key from attestation well known endpoint and create jwks object that will be used to verify the attestation tokens later in the demo. 

In [1]:

import subprocess
import json
import base64
import requests
import jwt
import json
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography import x509
from hashlib import sha256

# TODO: update to reflect your setup
registry_name = 'replace with your docker regitry name' # e.g. docker.io/pawankhandavillims
attestation_endpoint = 'sharedeus2.eus2.test.attest.azure.net' #this is a shared instance of MAA that you can use for testing
#runtime data is data you want reflected in the attestation token, this is not used in a meaningful way in this demo. 
runtime_data = 'eyJrZXlzIjpbeyJlIjoiQVFBQiIsImtleV9vcHMiOlsiZW5jcnlwdCJdLCJraWQiOiJOdmhmdXEyY0NJT0FCOFhSNFhpOVByME5QXzlDZU16V1FHdFdfSEFMel93Iiwia3R5IjoiUlNBIiwibiI6InY5NjVTUm15cDh6Ykc1ZU5GdURDbW1pU2VhSHB1akcyYkNfa2VMU3V6dkRNTE8xV3lyVUp2ZWFhNWJ6TW9PMHBBNDZwWGttYnFIaXNvelZ6cGlORExDbzZkM3o0VHJHTWVGUGYyQVBJTXUtUlNyek41NnF2SFZ5SXI1Y2FXZkhXay1GTVJEd0FlZnlOWVJIa2RZWWtnbUZLNDRoaFVkdGxDQUtFdjVVUXBGWmp2aDRpSTlqVkJkR1lNeUJhS1FMaGpJNVdJaC1RRzZaYTVzU3VPQ0ZNbm11eXV2TjVEZmxwTEZ6NTk1U3MtRW9CSVktTmlsNmxDdHZjR2dSLUlialVZSEFPczVhamFtVHpnZU84a3gzVkNFOUhjeUtteVVac2l5aUY2SURScDJCcHkzTkhUakl6N3Rta3BUSHg3dEhuUnRsZkUyRlV2MEI2aV9RWWxfWkE1USJ9XX0='

#extract the public key from the openid-configuration and create a JWKS object

def rsa_public_key_from_pem(cert_pem):
    cert = x509.load_pem_x509_certificate(cert_pem.encode(), default_backend())
    return cert.public_key()

response = requests.get(f"https://{attestation_endpoint}/certs")

if response.status_code == 200:
    cert_data = response.json()
    keys = cert_data['keys']

    # Step 2: Create a JWKS object
    jwks = []

    for key_data in keys:
        x5c = key_data.get('x5c', [])
        if x5c:
            cert_pem = "-----BEGIN CERTIFICATE-----\n" + x5c[0] + "\n-----END CERTIFICATE-----"
            public_key = rsa_public_key_from_pem(cert_pem)
            jwks.append((key_data['kid'], public_key))

    print("JWKS object created successfully.")
else:
    print("Failed to retrieve the signing keys.")

JWKS object created successfully.


### Step 2 : Build and upload container image to registry 

In this step we are going to build the "sum" container image that exposes a rest endpoint to calculate the sum of two numbers and also exposes an endpoint to get a MAA attestation token. 

In [ ]:
# build and push the image to the registry
#cacidemo is a sample repository name, you can change it to something else and update below
subprocess.run(f'docker build -t {registry_name}/cacidemo:latest ./sum', capture_output=True)

subprocess.run(f'docker push {registry_name}/cacidemo:latest', capture_output=True)

### Step 3 : Generate security policy for sum container 

We will use the confcom tooling to generate a security policy from the Azure Resource Manager template. We will further generate a SHA-256 hash of the security policy which will be used later in the demo to verify whether the container group is running the right configuration. 

Note : The "ccePolicy" attribute of the ARM template must be set to a null string "" for this step to work. The tooling requires user input to override the policy if already present and user input is not supported in the notebook. 

In [2]:
# generate security policy
subprocess.run('az confcom acipolicygen -a ./sum/template.json', capture_output=True, shell=True)

# get the hash of the security policy
with open("./sum/template.json", "r") as f:
    # open the template and grab the cce policy
    template = json.loads(f.read())
    security_policy = template.get('resources')[0]['properties']['confidentialComputeProperties']['ccePolicy']
    # decode the base64 encoded policy and hash it
    sha256_hash_sum = sha256(base64.b64decode(security_policy)).hexdigest()
    # print the hash
    print("hash of security policy: ", sha256_hash_sum)


hash of security policy:  670fff86714a650a49b58fadc1e90fedae0eb32dd51e34931c1e7a1839c08f6f


### Step 4 

Deploy the container group to ACI.

In [3]:
#deploy ARM Template 
# TODO : Need to update the resource group name to your own resource group name
subprocess.run('az deployment group create -g change_to_resource_group_name -f ./sum/template.json', capture_output=True, shell=True)

CompletedProcess(args='az deployment group create -g akhandavilli-rg -f ./sum/template.json', returncode=0, stdout=b'{\r\n  "id": "/subscriptions/eec8e14e-b47d-40d9-8bd9-23ff5c381b40/resourceGroups/akhandavilli-rg/providers/Microsoft.Resources/deployments/template",\r\n  "location": null,\r\n  "name": "template",\r\n  "properties": {\r\n    "correlationId": "d497a187-030e-4d4d-a9c0-9d117b949883",\r\n    "debugSetting": null,\r\n    "dependencies": [],\r\n    "duration": "PT1M11.8548777S",\r\n    "error": null,\r\n    "mode": "Incremental",\r\n    "onErrorDeployment": null,\r\n    "outputResources": [\r\n      {\r\n        "id": "/subscriptions/eec8e14e-b47d-40d9-8bd9-23ff5c381b40/resourceGroups/akhandavilli-rg/providers/Microsoft.ContainerInstance/containerGroups/aci-demo",\r\n        "resourceGroup": "akhandavilli-rg"\r\n      }\r\n    ],\r\n    "outputs": {\r\n      "containerIPv4Address": {\r\n        "type": "String",\r\n        "value": "20.93.85.149"\r\n      }\r\n    },\r\n    "

### Step 5 : Check for successful deployment on Azure Portal and get attestation token
In this step we will check for the successful deployment and get the attestation token verified by MAA. We will compare the contents of the "x-ms-sevsnpvm-hostdata" claim and check whether it matches the policy hash from step 3

In [6]:

# TODO: update the public_ip_address to the public ip address of your deployed container group. You can obtain the ip address from azure portal.  

public_ip_address = 'update to public ip address of your container group'
# call the maa endpoint
maa_response = requests.post(f'http://{public_ip_address}/attest/maa', 
                            json={"runtime_data": runtime_data, "maa_endpoint": attestation_endpoint})
print("Maa Response: ", maa_response.json())
token = json.loads(maa_response.json().get("result")).get("token")

# verify the token

header = jwt.get_unverified_header(token)
kid = header['kid']

# Find the key with a matching 'kid' in the JWKS
key_to_use = None
for key_kid, key in jwks:
        if key_kid == kid:
            key_to_use = key
            break

if key_to_use is not None:
        try:
            payload = jwt.decode(token, key=key_to_use, algorithms=["RS256"])
            print("Valid JWT : Attestation token signature verified:", payload)
            
        except jwt.InvalidTokenError:
            print("Invalid JWT")
else:
        print("No matching key found in JWKS")

print("SEV-SNP Host Data:\n", payload.get("x-ms-sevsnpvm-hostdata"))
if(sha256_hash_sum == payload.get("x-ms-sevsnpvm-hostdata")):
    print("Security Policy Hash Matches")
    print("Host is Trusted")
else:
    print("Security Policy Hash Does Not Match")
    print("Host is Not Trusted")

Maa Response:  {'result': '{"token":"eyJhbGciOiJSUzI1NiIsImprdSI6Imh0dHBzOi8vc2hhcmVkZXVzMi5ldXMyLnRlc3QuYXR0ZXN0LmF6dXJlLm5ldC9jZXJ0cyIsImtpZCI6IjNiZENZSmFiemZoSVNGdGIzSjh5dUVFU1p3dWZWN2hoaDA4TjNaZmxBdUU9IiwidHlwIjoiSldUIn0.eyJleHAiOjE2ODAyNTk5OTcsImlhdCI6MTY4MDIzMTE5NywiaXNzIjoiaHR0cHM6Ly9zaGFyZWRldXMyLmV1czIudGVzdC5hdHRlc3QuYXp1cmUubmV0IiwianRpIjoiZDI4OGZlZjU4ODBiMTUwMWVhNzBiZTFiOTM2Njg0MGZkNTZmNzRlNjY2YTIzMjI0ZDZkZTExMzEzM2NiZDhkNSIsIm5iZiI6MTY4MDIzMTE5Nywibm9uY2UiOiIzNDEzNzY0MDQ5MDA1MjcwMTM5IiwieC1tcy1hdHRlc3RhdGlvbi10eXBlIjoic2V2c25wdm0iLCJ4LW1zLWNvbXBsaWFuY2Utc3RhdHVzIjoiYXp1cmUtY29tcGxpYW50LXV2bSIsIngtbXMtcG9saWN5LWhhc2giOiI5TlkwVm5UUS1JaUJyaUJwbFZVcEZiY3pjRGFFQlV3c2lGWUF6SHVfZ2NvIiwieC1tcy1ydW50aW1lIjp7ImtleXMiOlt7ImUiOiJBUUFCIiwia2V5X29wcyI6WyJlbmNyeXB0Il0sImtpZCI6Ik52aGZ1cTJjQ0lPQUI4WFI0WGk5UHIwTlBfOUNlTXpXUUd0V19IQUx6X3ciLCJrdHkiOiJSU0EiLCJuIjoidjk2NVNSbXlwOHpiRzVlTkZ1RENtbWlTZWFIcHVqRzJiQ19rZUxTdXp2RE1MTzFXeXJVSnZlYWE1YnpNb08wcEE0NnBYa21icUhpc296VnpwaU5ETENvNmQzejRUckdNZUZ

### Step 6 : Error scenario : Attempt to deploy an image different than what is captured in the security policy 
We want to break the deployment by deploying a different image, in this case a container image that calculates the product of two numbers instead of the sum with the same repository and tag.

After the successful execution of this step, please go to the azure portal and restart the container group that was deployed previously in Step 4. Once the container group restarts, the new image will break the policy and fail the deployment as the updated image does not reflect the policy. This is simulating an attack scenario where the container image is updated by a malicious actor. 
This is also why the `latest` tag is not recommended for Confidential ACI. 


In [ ]:
# deploy product + skr container with ARM Template from CLI
#cacidemo is a sample repository name, you can change it to something else and update below. For this scenario the name should be the same as the one used in step2

subprocess.run(f'docker build -t {registry_name}/cacidemo:latest ./product', capture_output=True)

subprocess.run(f'docker push {registry_name}/cacidemo:latest', capture_output=True)

### Step 7 : Generate security policy for product container 

We will use the confcom tooling to generate a security policy from the Azure Resource Manager template.

Note : The "ccePolicy" attribute of the ARM template must be set to a null string "" for this step to work. The tooling requires user input to override the policy if already present and user input is not supported in the notebook. 

In [ ]:
# generate security policy
subprocess.run('az confcom acipolicygen -a ./product/template.json', capture_output=True, shell=True)

# get the hash of the security policy
with open("./product/template.json", "r") as f:
    # open the template and grab the cce policy
    template = json.loads(f.read())
    security_policy = template.get('resources')[0]['properties']['confidentialComputeProperties']['ccePolicy']
    # decode the base64 encoded policy and hash it
    sha256_hash_sum = sha256(base64.b64decode(security_policy)).hexdigest()
    # print the hash
    print("hash of security policy: ", sha256_hash_sum)

### Step 8 : Deploy updated container group with the updated security policy 
We want to fix our deployment by generating a new security policy to reflect the updated image, then uploading the ARM Template with the new policy.

In [ ]:
# deploy ARM Template
# TODO : Need to update the resource group name to your own resource group name

subprocess.run('az deployment group create -g akhandavilli-rg -f ./product/template.json', shell=True)

### Step 9 : Check deployment is successful and get attestation token 
In this step the security policy hash as part of the attestation token will not match the one generated by the tooling in step 1 as the image has changed. 

In [ ]:
# TODO: update the public_ip_address from the Azure portal 

public_ip_address = 'ip address of your deployed container group here'

# call the maa endpoint
maa_response = requests.post(f'http://{public_ip_address}/attest/maa', 
                            json={"runtime_data": runtime_data, "maa_endpoint": attestation_endpoint})

token = json.loads(maa_response.json().get("result")).get("token")

# verify the token

header = jwt.get_unverified_header(token)
kid = header['kid']

# Find the key with a matching 'kid' in the JWKS
key_to_use = None
for key_kid, key in jwks:
        if key_kid == kid:
            key_to_use = key
            break

if key_to_use is not None:
        try:
            payload = jwt.decode(token, key=key_to_use, algorithms=["RS256"])
            print("JWT is valid:", payload)
        except jwt.InvalidTokenError:
            print("Invalid JWT")
else:
        print("No matching key found in JWKS")

print("SEV-SNP Host Data:\n", payload.get("x-ms-sevsnpvm-hostdata"))
if(sha256_hash_sum == payload.get("x-ms-sevsnpvm-hostdata")):
    print("Security Policy Hash Matches")
    print("Host is Trusted")
else:
    print("Security Policy Hash Does Not Match")
    print("Host is Not Trusted")
